In [3]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
df1 = pd.read_csv(r"D:\desktop\B\数据\标准化后数据\df1.csv")
df2 = pd.read_excel(r"D:\desktop\B\数据\标准化后数据\df2.xlsx")
df1[['mort', 'CONS','MV','CRRT']] = df1[['mort',  'CONS','MV','CRRT']].astype('category')
df2[['mort',  'CONS','MV','CRRT']] = df2[['mort',  'CONS','MV','CRRT']].astype('category')
selected_features = ["CONS", "LDH", "MV", "AST", "CRRT", "U", "L", "HR", "D.Dimer", "CR", "age", "ALT"]
target = 'mort'
X = df1[selected_features]
y = df1[target]
X_train, X_internal_test, y_train, y_internal_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_external_test = df2[selected_features]
y_external_test = df2[target]

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import resample

RANDOM_SEED = 42

# 定义计算置信区间的函数
def bootstrap_ci(y_true, y_pred, metric_func, n_bootstraps=10000, ci=95, random_state=42):
    rng = np.random.RandomState(random_state)
    bootstrapped_scores = []
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), replace=True, random_state=rng)
        score = metric_func(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    
    lower_bound = np.percentile(bootstrapped_scores, (100-ci)/2)
    upper_bound = np.percentile(bootstrapped_scores, 100-(100-ci)/2)
    return lower_bound, upper_bound


params_gbd = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.1, 0.31, 0.5, 0.7],
    'max_depth': [1, 3, 5, 7],
    'criterion': ['friedman_mse', 'squared_error']
}

best_models_gbd = {}

for i in range(1, len(selected_features) + 1):
    current_features = selected_features[:i]
    X_train_current = X_train[current_features]
    X_internal_test_current = X_internal_test[current_features]
    X_external_test_current = X_external_test[current_features]
    
    model_gbd = GradientBoostingClassifier(random_state=RANDOM_SEED)
    grid_gbd = GridSearchCV(model_gbd, params_gbd, cv=10, scoring='roc_auc', n_jobs=-1)
    grid_gbd.fit(X_train_current, y_train)
    
    best_model_gbd = grid_gbd.best_estimator_

    # 使用交叉验证在训练集和内部验证集上进行预测
    y_train_pred_cv = cross_val_predict(best_model_gbd, X_train_current, y_train, cv=10, method='predict')
    y_train_proba_cv = cross_val_predict(best_model_gbd, X_train_current, y_train, cv=10, method='predict_proba')[:, 1]
    
    y_internal_pred_cv = cross_val_predict(best_model_gbd, X_internal_test_current, y_internal_test, cv=10, method='predict')
    y_internal_proba_cv = cross_val_predict(best_model_gbd, X_internal_test_current, y_internal_test, cv=10, method='predict_proba')[:, 1]

    # 在外部验证集上进行预测
    y_external_pred_gbd = best_model_gbd.predict(X_external_test_current)
    y_external_proba_gbd = best_model_gbd.predict_proba(X_external_test_current)[:, 1]

    # 计算训练集的各项指标分数
    train_accuracy = accuracy_score(y_train, y_train_pred_cv)
    train_precision = precision_score(y_train, y_train_pred_cv, average='weighted', zero_division=0)
    train_recall = recall_score(y_train, y_train_pred_cv, average='weighted')
    train_f1 = f1_score(y_train, y_train_pred_cv, average='weighted')
    train_auc = roc_auc_score(y_train, y_train_proba_cv)

    # 计算训练集的各项指标的置信区间
    train_ci = {
        'accuracy_ci': bootstrap_ci(y_train, y_train_pred_cv, accuracy_score),
        'precision_ci': bootstrap_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_ci(y_train, y_train_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_ci(y_train, y_train_proba_cv, roc_auc_score)
    }

    # 计算内部验证集的各项指标分数
    internal_accuracy = accuracy_score(y_internal_test, y_internal_pred_cv)
    internal_precision = precision_score(y_internal_test, y_internal_pred_cv, average='weighted', zero_division=0)
    internal_recall = recall_score(y_internal_test, y_internal_pred_cv, average='weighted')
    internal_f1 = f1_score(y_internal_test, y_internal_pred_cv, average='weighted')
    internal_auc = roc_auc_score(y_internal_test, y_internal_proba_cv)

    # 计算内部验证集的各项指标的置信区间
    internal_ci = {
        'accuracy_ci': bootstrap_ci(y_internal_test, y_internal_pred_cv, accuracy_score),
        'precision_ci': bootstrap_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_ci(y_internal_test, y_internal_pred_cv, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_ci(y_internal_test, y_internal_proba_cv, roc_auc_score)
    }

    # 计算外部验证集的各项指标分数
    external_accuracy = accuracy_score(y_external_test, y_external_pred_gbd)
    external_precision = precision_score(y_external_test, y_external_pred_gbd, average='weighted', zero_division=0)
    external_recall = recall_score(y_external_test, y_external_pred_gbd, average='weighted')
    external_f1 = f1_score(y_external_test, y_external_pred_gbd, average='weighted')
    external_auc = roc_auc_score(y_external_test, y_external_proba_gbd)

    # 计算外部验证集的各项指标的置信区间
    external_ci = {
        'accuracy_ci': bootstrap_ci(y_external_test, y_external_pred_gbd, accuracy_score),
        'precision_ci': bootstrap_ci(y_external_test, y_external_pred_gbd, lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted', zero_division=0)),
        'recall_ci': bootstrap_ci(y_external_test, y_external_pred_gbd, lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted')),
        'f1_score_ci': bootstrap_ci(y_external_test, y_external_pred_gbd, lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')),
        'auc_ci': bootstrap_ci(y_external_test, y_external_proba_gbd, roc_auc_score)
    }

    best_models_gbd[i] = {
        'features': current_features,
        'model': best_model_gbd,  
        'train_metrics': {
            'accuracy': train_accuracy,
            'precision': train_precision,
            'recall': train_recall,
            'f1_score': train_f1,
            'auc': train_auc
        },
        'train_ci': train_ci,
        'internal_metrics': {
            'accuracy': internal_accuracy,
            'precision': internal_precision,
            'recall': internal_recall,
            'f1_score': internal_f1,
            'auc': internal_auc
        },
        'internal_ci': internal_ci,
        'external_metrics': {
            'accuracy': external_accuracy,
            'precision': external_precision,
            'recall': external_recall,
            'f1_score': external_f1,
            'auc': external_auc
        },
        'external_ci': external_ci,
        'best_params': grid_gbd.best_params_
    }

# 输出每个模型的结果
for i, results in best_models_gbd.items():
    print(f"\nModel with top {i} features: {results['features']}")
    print("Best Parameters:", results['best_params'])

    print("Training Set Metrics:")
    for metric, value in results['train_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}")

    print("Internal Validation Set Metrics:")
    for metric, value in results['internal_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}")
    
    print("External Validation Set Metrics:")
    for metric, value in results['external_metrics'].items():
        print(f"{metric.capitalize()}: {value:.4f} 置信区间: {results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}")



Model with top 1 features: ['CONS']
Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 50}
Training Set Metrics:
Accuracy: 0.8676 置信区间: 0.8186-0.9118
Precision: 0.8760 置信区间: 0.8312-0.9185
Recall: 0.8676 置信区间: 0.8186-0.9118
F1_score: 0.8706 置信区间: 0.8235-0.9134
Auc: 0.7913 置信区间: 0.7019-0.8720
Internal Validation Set Metrics:
Accuracy: 0.7045 置信区间: 0.6136-0.7955
Precision: 0.7292 置信区间: 0.6319-0.8282
Recall: 0.7045 置信区间: 0.6136-0.7955
F1_score: 0.7137 置信区间: 0.6200-0.8019
Auc: 0.6719 置信区间: 0.5297-0.8044
External Validation Set Metrics:
Accuracy: 0.8558 置信区间: 0.7885-0.9231
Precision: 0.8503 置信区间: 0.7767-0.9215
Recall: 0.8558 置信区间: 0.7885-0.9231
F1_score: 0.8476 置信区间: 0.7696-0.9183
Auc: 0.7628 置信区间: 0.6643-0.8610

Model with top 2 features: ['CONS', 'LDH']
Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.7, 'max_depth': 1, 'n_estimators': 50}
Training Set Metrics:
Accuracy: 0.8529 置信区间: 0.8039-0.8971
Precision: 0.8478 置信区间: 0

In [5]:
import pandas as pd
import os

# 设置输出文件夹路径
output_folder = r"D:\desktop\B\结果\表格\内部外部验证结果"
os.makedirs(output_folder, exist_ok=True)  # 确保输出文件夹存在

# 用于存储结果的列表
output_data = []

# 遍历模型结果并将其组织为字典
for i, results in best_models_gbd.items():
    model_name = f"Model with top {i} features"
    for metric in results['train_metrics'].keys():
        output_data.append({
            'Model': model_name,
            'Dataset': 'Training Set',
            'Metric': metric.capitalize(),
            'Value': results['train_metrics'][metric],
            'Confidence Interval': f"{results['train_ci'][metric+'_ci'][0]:.4f}-{results['train_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'Internal Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['internal_metrics'][metric],
            'Confidence Interval': f"{results['internal_ci'][metric+'_ci'][0]:.4f}-{results['internal_ci'][metric+'_ci'][1]:.4f}"
        })
        output_data.append({
            'Model': model_name,
            'Dataset': 'External Validation Set',
            'Metric': metric.capitalize(),
            'Value': results['external_metrics'][metric],
            'Confidence Interval': f"{results['external_ci'][metric+'_ci'][0]:.4f}-{results['external_ci'][metric+'_ci'][1]:.4f}"
        })

# 将列表转换为 DataFrame
output_df = pd.DataFrame(output_data)

# 设置文件名
output_file = os.path.join(output_folder, "GBD_Results.xlsx")

# 将 DataFrame 保存为 Excel 文件
output_df.to_excel(output_file, index=False)

print(f"结果已保存至：{output_file}")


结果已保存至：D:\desktop\B\结果\表格\内部外部验证结果\GBD_Results.xlsx
